In [718]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime as dt, date, timedelta
from polygon import RESTClient
from secret import API_KEY

DATA_DIR = 'data'
CSV_DIR = 'csv'
PRICES_DIR = 'prices'
CHAINS_DIR = 'chains'

In [719]:
# Getting API key and client
api_key = API_KEY
client = RESTClient(api_key)

In [720]:
# Importing buy/sell signals
read_path = os.path.join(DATA_DIR, CSV_DIR, 'signals.csv')
data = pd.read_csv(read_path)

In [721]:
# Creating columns for prices purchased and sold at 
data['call_prices'] = np.zeros(len(data))
data['put_prices'] = np.zeros(len(data))

In [722]:
# Setting ticker
underlying_ticker = 'SPY'

In [723]:
def get_option_ticker(timestamp, price, type):
    # Fixing timestamps
    date = pd.to_datetime(timestamp).date() + timedelta(weeks=1)
    date_more = date + timedelta(weeks=1)

    save_ticker_path = os.path.join(DATA_DIR, CHAINS_DIR, f'{timestamp}_{price}_{type}.csv')

    if os.path.exists(save_ticker_path):
        df = pd.read_csv(save_ticker_path)
        option_ticker = df['ticker'].iloc[0]

        return option_ticker
    
    else:
        
        try:
            # Initialize a list to collect all contract data
            options_list = []

            # Fetch options data
            options_data = client.list_options_contracts(
                underlying_ticker=underlying_ticker,
                expiration_date_lte=date_more,
                expiration_date_gte=date,
                strike_price_gte=(price),
                strike_price_lte=(price+1),
                contract_type=type,
                limit=1000,
                expired=True
            )

            # Loop through the options data and append each contract's data to options_list
            for option in options_data:
                options_list.append(option.__dict__)

            # Convert the list of options data to a DataFrame
            df = pd.DataFrame(options_list)

            # Finding closest ticker
            option_ticker = df['ticker'].iloc[0]

            df.to_csv(save_ticker_path, index=False)

            return option_ticker
        
        except Exception as e:
            print(f'Ticker Error: {e}')

    return None
    

def get_option_data(timestamp, ticker):
    # Fixing date
    date = pd.to_datetime(timestamp).date()

    try:
        # Downloading price data
        aggs = client.get_aggs(
                ticker=ticker,
                multiplier=1,
                timespan="minute",
                from_=date,
                to=date,
                limit=5000
            )
        
        # Converting to DataFrame
        df = pd.DataFrame([agg.__dict__ for agg in aggs])

        # Fixing dates again
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')
        
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)

        # Filling in missing data
        df = df.resample('1T').ffill()

        # Resetting indx and creating time column
        df.reset_index(inplace=True, drop=False)
        df['time'] = df['timestamp'].dt.time
        df['time'] = df['time'].apply(lambda t: t.strftime("%H:%M:%S"))
        
        return df
    except Exception as e:
        print(f"Error retreieving data: {e}")

    return None

def retrieve_price_data(row, type):
    timestamp = row['timestamp']
    ticker = get_option_ticker(timestamp, row['close'], type)
    date = row['date']
    
    save_data_path = os.path.join(DATA_DIR, PRICES_DIR, f'{date}_{ticker}.csv')
    
    if os.path.exists(save_data_path):
        option_price_data = pd.read_csv(save_data_path)
    else:
        option_price_data = get_option_data(timestamp, ticker)
        
        try:
            option_price_data.to_csv(save_data_path, index=False)
        except Exception as e:
            print(f'Price Error: {e}')

    return option_price_data

def find_stop_loss(df, buy_time, buy_price):
    stop_loss = buy_price * 0.9

    time_index = df[df['timestamp'] == buy_time].index[0]
    df = df[time_index:]

    for index, row in df.iterrows():
        if stop_loss > row['close']:
            stop_loss_time = row['timestamp']
            stop_loss_time = stop_loss_time[11:19]
            break
        else:
            stop_loss_time = '24:00:00'
    
    return stop_loss, stop_loss_time


In [ ]:
# Resetting variables
buy_call_price = None
buy_put_price = None
sell_call_price = None
sell_put_price = None
option_price_data = None
data = data.iloc[:-14]

for index, row in data.iterrows():
    """
    Looping through all rows of sorted data with buy signals
    1. Determines action
    2. Finds timestamp
    3. Finds relevant option ticker
    4. Checks if previous data has been saved
    5. Gets option price data
    6. Finds option price at buy time
    Note: some option prices had to be interpolated due to some missing data from source of options data
    7. Record price at actions
    """
    if row['call_buy'] and row['call_prices'] == 0:
        option_price_data = retrieve_price_data(row, 'call')
        row_time = pd.to_datetime(row['time']).strftime("%H:%M:%S")

        try:
            buy_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
            buy_call_time = (option_price_data[option_price_data['time'] == row_time])['timestamp'].iloc[0]
            data.at[index, 'call_prices'] = buy_call_price

            print(index)

        except Exception as e:
            print(f'Error: {e}')
            data.at[index, 'call_prices'] = None
            continue

    elif row['call_sell'] and row['call_prices'] == 0:
        option_price_data = retrieve_price_data(row, 'call')
        row_time = pd.to_datetime(row['time']).strftime("%H:%M:%S")

        try:
            stop_loss, stop_loss_time = find_stop_loss(option_price_data, buy_call_time, buy_call_price)

            if stop_loss_time < row_time:
                sell_call_price = stop_loss
            elif stop_loss_time > row_time:
                sell_call_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
            
            data.at[index, 'call_prices'] = sell_call_price
            print(index)

        except Exception as e:
            print(f'Error: {e}')
            data.at[index, 'call_prices'] = None
            continue

    elif row['put_buy'] and row['put_prices'] == 0:
        option_price_data = retrieve_price_data(row, 'put')
        row_time = pd.to_datetime(row['time']).strftime("%H:%M:%S")

        try:
            buy_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]
            buy_put_time = (option_price_data[option_price_data['time'] == row_time])['timestamp'].iloc[0]
            data.at[index, 'put_prices'] = buy_put_price
            
            print(index)
            
        except Exception as e:
            print(f'Error: {e}')
            data.at[index, 'put_prices'] = None
            continue

    elif row['put_sell'] and row['put_prices'] == 0:
        option_price_data = retrieve_price_data(row, 'put')
        row_time = pd.to_datetime(row['time']).strftime("%H:%M:%S")        

        try:
            stop_loss, stop_loss_time = find_stop_loss(option_price_data, buy_put_time, buy_put_price)

            if stop_loss_time < row_time:
                sell_put_price = stop_loss
            elif stop_loss_time > row_time:
                sell_put_price = (option_price_data[option_price_data['time'] == row_time])['close'].iloc[0]

            
            data.at[index, 'put_prices'] = sell_put_price
            print(index)

        except Exception as e:
            print(f'Error: {e}')
            data.at[index, 'put_prices'] = None
            continue

0
True
1
2
False
3
4
False
5
6
False
7
8
False
9
10
False
11
12
False
13
14
15
16
17
18
True
19
20
False
21
22
False
23
24
25
26
False
27
28
29
30
31
32
33
34
False
35
36
37
38
False
39
40
41
42
43
44
False
45
46
47
48
49
50
False
51
52
False
53
54
False
55
Error: single positional indexer is out-of-bounds
Error: index 0 is out of bounds for axis 0 with size 0
58
False
59
60
61
62
False
63
64
False
65
66
False
67
68
69
70
71
72
73
74
False
75
76
False
77
78
False
79
80
81
82
False
83
84
False
85
86
False
87
88
89
90
91
92
93
94
95
96
97
98
99
100
False
101
102
False
103
104
105
106
False
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
False
129
130
False
131
132
False
133
134
135
136
False
137
138
139
140
141
142
False
143
144
False
145
146
True
147
148
149
150
False
151
152
False
153
154
False
155
156
False
157
158
False
159
160
161
162
163
164
165
166
167
168
169
170
171
172
False
173
174
False
175
176
True
177
178
False
179
180
False
181
182
1

In [725]:
# Backing up data
save_path = os.path.join(DATA_DIR, CSV_DIR, 'backup_data.csv')
data.to_csv(save_path)

In [726]:
# Creating list of index values associated with null data
null_list = (data[(data.isna()['call_prices'] | data.isna()['put_prices']) == True].index).to_list()

# Iterating through rows and remvoing trades consisting of null data
for row_index in null_list:
    if row_index % 2 == 0:
        data = (data.drop([row_index, (row_index+1)])).copy()
        try:
            null_list.remove(row_index+1)
        except:
            continue
    else:
        data = (data.drop([(row_index -1), row_index])).copy()

In [727]:
# Saving to csv
save_path = os.path.join(DATA_DIR, CSV_DIR, 'backtest_data.csv')
data.to_csv(save_path, index=False)
data = None